In [ ]:
# default_exp xml

# xml

> Functionality to parse XML files

In [ ]:
# export
import pathlib
import re
import datetime

import pandas as pd
from lxml import etree

Directory where the data (*XML* files) are stored

In [ ]:
directory = pathlib.Path.cwd() / 'samples'
assert directory.exists()
directory

PosixPath('/home/manu/dlsproc/samples')

A (sample) file in that directory

In [ ]:
# xml_file = directory / 'sample.xml'
xml_file = directory / 'PlataformasAgregadasSinMenores_20220104_030016_1.atom'
assert xml_file.exists()
xml_file

PosixPath('/home/manu/dlsproc/samples/PlataformasAgregadasSinMenores_20220104_030016_1.atom')

*Root* element of the *XML* tree

In [ ]:
root = etree.parse(xml_file).getroot()

## Convenience functions

A function to extract the *namespace*s declared in an *XML* file

In [ ]:
# export
def get_namespaces(input_file: str | pathlib.Path, root_name: str = 'base') -> dict:
    
    tree = etree.parse(input_file)
    
    namespaces = tree.getroot().nsmap
    
    if None in namespaces:
        
        namespaces[root_name] = namespaces.pop(None)
        
    return namespaces

In [ ]:
get_namespaces(xml_file)

{'cbc-place-ext': 'urn:dgpe:names:draft:codice-place-ext:schema:xsd:CommonBasicComponents-2',
 'cac-place-ext': 'urn:dgpe:names:draft:codice-place-ext:schema:xsd:CommonAggregateComponents-2',
 'cbc': 'urn:dgpe:names:draft:codice:schema:xsd:CommonBasicComponents-2',
 'cac': 'urn:dgpe:names:draft:codice:schema:xsd:CommonAggregateComponents-2',
 'ns1': 'urn:oasis:names:specification:ubl:schema:xsd:CommonExtensionComponents-2',
 'base': 'http://www.w3.org/2005/Atom'}

In order to trim off *namespace*s from a tag:
- a regular expression

In [ ]:
# export
re_tag = re.compile('\{(.*)\}(.*)')

In [ ]:
assert re_tag.match('{blabla}foo').groups() == ('blabla', 'foo')

In [ ]:
re_tag.match('{some.namespace}id').groups()

('some.namespace', 'id')

* a function

In [ ]:
# export
def split_namespace_tag(namespace_tag: str) -> str:
    return re_tag.match(namespace_tag).groups()

In [ ]:
split_namespace_tag('{some.namespace}id')

('some.namespace', 'id')

In [ ]:
split_namespace_tag(root.tag)

('http://www.w3.org/2005/Atom', 'feed')

In [ ]:
# export
to_be_skipped = ['author', 'id', 'link', 'title', 'updated', r'deleted-entry']
to_be_skipped

['author', 'id', 'link', 'title', 'updated', 'deleted-entry']

A function to get list of `etree.Element` with all the *entries* (allegedly, *procurement contracts*)

In [ ]:
# export
def get_entries(root: etree.Element) -> list[etree.Element]:
    
    return [e for e in etree.parse(xml_file).getroot() if split_namespace_tag(e.tag)[1] == 'entry']

*Entries* are extracted using the above function (only the 4 first ones are shown)

In [ ]:
entries = get_entries(root)
assert len(entries) == 117
entries[:4]

[<Element {http://www.w3.org/2005/Atom}entry at 0x7f12fbcb5640>,
 <Element {http://www.w3.org/2005/Atom}entry at 0x7f12fbcb5680>,
 <Element {http://www.w3.org/2005/Atom}entry at 0x7f12fbcb4fc0>,
 <Element {http://www.w3.org/2005/Atom}entry at 0x7f12fbcb56c0>]

In [ ]:
element = entries[0]
split_namespace_tag(element.tag)

('http://www.w3.org/2005/Atom', 'entry')

In [ ]:
subelement = element[3]
split_namespace_tag(subelement.tag)

('http://www.w3.org/2005/Atom', 'title')

In [ ]:
subelement.text

"L'objecte del contracte és la renovació de totes les llumeneres que formen la il·luminació existent de tots els carrers i vials del casc urbà de la localitat de Sant Ramon i dels nuclis agregats de La Manresana, Portell, Viver i Gospí"

In [ ]:
element

<Element {http://www.w3.org/2005/Atom}entry at 0x7f12fbcb5640>

Everything is stored as a string in an *XML* file. The approach below is used to handle conversions.

In [ ]:
numeric_field = '8'
numeric_field.isnumeric()
float(numeric_field).is_integer()

True

When two a *tag* is nested inside other *tag* the string below is used to assemble the name of the resulting *column* from those of the original tags

In [ ]:
# export
nested_tags_separator = ' - '

For instance, a *tag* `<month>` inside a *tag* `<date>` will yield a *column*

In [ ]:
f'month{nested_tags_separator}date'

'month - date'

A **recursive** function to parse a node of the *XML* tree

In [ ]:
# export
def entry_to_dict(entry: etree.Element, recursive: bool = True) -> dict:

    res = {}
    
    # for every "child" of `entry` ...
    for e in entry:
        
        # ...the *namespace* and *tag* are extracted
        namespace, tag = split_namespace_tag(e.tag)
        
        # for the sake of readability
        value = e.text
            
        # if `text` is "something" and not an empty string after striping it of blank characteres...
        if value and (value.strip() != ''):
            
            # if the text contains a number...
            if value.isnumeric():
                
                # ...it is turned into a `float`
                value = float(value)
                
                # if the latter is actually an integer...
                if value.is_integer():
                    
                    # ...conversion is performed
                    value = int(value)

            # the value of this element (whether the original text or the obtained number) is stored
            res[tag] = value
        
        # if in "recursive mode" and this element has children (`len(e)` is different from 0)...
        if recursive and len(e):
            
            # recursion
            sub_res = entry_to_dict(e)
            
            for k, v in sub_res.items():
                
                # the name of the new "key" is assembled from those of the parent and the child
                res[f'{tag}{nested_tags_separator}{k}'] = v
    
    return res

In [ ]:
element_series = entry_to_dict(element)
element_series

{'id': 'https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/8799346',
 'summary': 'Id licitación: C. 2-2021; Órgano de Contratación: Ajuntament de Sant Ramon; Importe: 135553.26; Estado: ADJUDICADA',
 'title': "L'objecte del contracte és la renovació de totes les llumeneres que formen la il·luminació existent de tots els carrers i vials del casc urbà de la localitat de Sant Ramon i dels nuclis agregats de La Manresana, Portell, Viver i Gospí",
 'updated': '2022-01-03T01:11:41.826+01:00',
 'ContractFolderStatus - ContractFolderID': 'C. 2-2021',
 'ContractFolderStatus - ContractFolderStatusCode': 'ADJ',
 'ContractFolderStatus - LocatedContractingParty - BuyerProfileURIID': 'https://contractaciopublica.gencat.cat/ecofin_pscp/AppJava/cap.pscp?reqCode=viewDetail&idCap=2763318',
 'ContractFolderStatus - LocatedContractingParty - Party - PartyName - Name': 'Ajuntament de Sant Ramon',
 'ContractFolderStatus - LocatedContractingParty - ParentLocatedParty - PartyName - Na

A function that just wraps the result of `entry_to_dict` into a `pd.Series`

In [ ]:
# export
def entry_to_series(entry: etree.Element) -> pd.Series:

    return pd.Series(entry_to_dict(entry))

Only the first 8 fields are printed (enough to show *nested* elements)

In [ ]:
element_series = entry_to_series(element)
element_series[:8]

id                                                                           https://contrataciondelestado.es/sindicacion/P...
summary                                                                      Id licitación: C. 2-2021; Órgano de Contrataci...
title                                                                        L'objecte del contracte és la renovació de tot...
updated                                                                                          2022-01-03T01:11:41.826+01:00
ContractFolderStatus - ContractFolderID                                                                              C. 2-2021
ContractFolderStatus - ContractFolderStatusCode                                                                            ADJ
ContractFolderStatus - LocatedContractingParty - BuyerProfileURIID           https://contractaciopublica.gencat.cat/ecofin_...
ContractFolderStatus - LocatedContractingParty - Party - PartyName - Name                             Ajuntamen

We can concatenate together the `pd.Series` for the different *entries* into a `pd.DataFrame`

In [ ]:
df = pd.concat([entry_to_series(e) for e in entries[:4]], axis=1).T
df

,id,summary,title,updated,ContractFolderStatus - ContractFolderID,ContractFolderStatus - ContractFolderStatusCode,ContractFolderStatus - LocatedContractingParty - BuyerProfileURIID,ContractFolderStatus - LocatedContractingParty - Party - PartyName - Name,ContractFolderStatus - LocatedContractingParty - ParentLocatedParty - PartyName - Name,ContractFolderStatus - ProcurementProject - Name,...,ContractFolderStatus - TenderResult - WinningParty - PartyName - Name,ContractFolderStatus - TenderResult - AwardedTenderedProject - LegalMonetaryTotal - TaxExclusiveAmount,ContractFolderStatus - TenderingProcess - ProcedureCode,ContractFolderStatus - TenderingProcess - TenderSubmissionDeadlinePeriod - EndDate,ContractFolderStatus - TenderingProcess - TenderSubmissionDeadlinePeriod - EndTime,ContractFolderStatus - ValidNoticeInfo - NoticeTypeCode,ContractFolderStatus - ValidNoticeInfo - AdditionalPublicationStatus - PublicationMediaName,ContractFolderStatus - ValidNoticeInfo - AdditionalPublicationStatus - AdditionalPublicationDocumentReference - IssueDate,ContractFolderStatus - LegalDocumentReference - ID,ContractFolderStatus - LegalDocumentReference - Attachment - ExternalReference - URI
0,https://contrataciondelestado.es/sindicacion/P...,Id licitación: C. 2-2021; Órgano de Contrataci...,L'objecte del contracte és la renovació de tot...,2022-01-03T01:11:41.826+01:00,C. 2-2021,ADJ,https://contractaciopublica.gencat.cat/ecofin_...,Ajuntament de Sant Ramon,Entitats municipals de Catalunya,L'objecte del contracte és la renovació de tot...,...,"AERONAVAL DE CONSTRUCCIONES I INSTALACIONES , ...",90078.51,9,2021-12-17,14:00:00,DOC_CAN_ADJ,Perfil del contratante,2022-01-03,NaN,NaN
1,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 8128_3/2021; Órgano de Contrata...,Obras de restauración hidromorfológica del río...,2022-01-03T01:00:11.194+01:00,8128_3/2021,PUB,NaN,Pleno del Ayuntamiento,AYUNTAMIENTO DE MONREAL,Obras de restauración hidromorfológica del río...,...,NaN,NaN,1,2022-01-22,23:30:00,DOC_CN,Perfil del contratante,2022-01-03,NaN,NaN
2,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 1000_0005-CP01-2021-000063; Órg...,Contrato del servicio de realización de labore...,2022-01-03T01:00:10.399+01:00,1000_0005-CP01-2021-000063,EV,NaN,El Director General de Comunicación y Relacion...,"Departamento de Presidencia, Igualdad, Función...",Contrato del servicio de realización de labore...,...,NaN,NaN,1,NaN,NaN,DOC_CN,Perfil del contratante,2022-01-03,NaN,NaN
3,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 1379/2020 4738; Órgano de Contr...,Obres de renovació de l'enllumenat públic a la...,2022-01-03T00:11:40.740+01:00,1379/2020 4738,EV,https://contractaciopublica.gencat.cat/ecofin_...,Ajuntament de Canet de Mar,Entitats municipals de Catalunya,Obres de renovació de l'enllumenat públic a la...,...,NaN,NaN,9,2022-01-02,23:59:00,DOC_CN,Perfil del contratante,2021-12-13,Plec Clausules.pdf,https://contractaciopublica.gencat.cat/ecofin_...


The types of the columns

In [ ]:
df.dtypes

id                                                                                                                           object
summary                                                                                                                      object
title                                                                                                                        object
updated                                                                                                                      object
ContractFolderStatus - ContractFolderID                                                                                      object
ContractFolderStatus - ContractFolderStatusCode                                                                              object
ContractFolderStatus - LocatedContractingParty - BuyerProfileURIID                                                           object
ContractFolderStatus - LocatedContractingParty - Party - PartyName - Name   

The function below follows the above strategy to turn an *XML* file into a *Pandas* `pd.DataFrame`

In [ ]:
# export
def to_df(input_file: str | pathlib.Path) -> pd.DataFrame:
    
    tree = etree.parse(input_file)
    root = tree.getroot()
    entries = get_entries(root)
    
    return pd.concat([entry_to_series(e) for e in entries], axis=1).T

In [ ]:
df = to_df(xml_file)
df

,id,summary,title,updated,ContractFolderStatus - ContractFolderID,ContractFolderStatus - ContractFolderStatusCode,ContractFolderStatus - LocatedContractingParty - BuyerProfileURIID,ContractFolderStatus - LocatedContractingParty - Party - PartyName - Name,ContractFolderStatus - LocatedContractingParty - ParentLocatedParty - PartyName - Name,ContractFolderStatus - ProcurementProject - Name,...,ContractFolderStatus - LegalDocumentReference - Attachment - ExternalReference - URI,ContractFolderStatus - TechnicalDocumentReference - ID,ContractFolderStatus - TechnicalDocumentReference - Attachment - ExternalReference - URI,ContractFolderStatus - ProcurementProject - PlannedPeriod - StartDate,ContractFolderStatus - ProcurementProject - PlannedPeriod - EndDate,ContractFolderStatus - LocatedContractingParty - Party - PartyIdentification - ID,ContractFolderStatus - LocatedContractingParty - ParentLocatedParty - ParentLocatedParty - PartyName - Name,ContractFolderStatus - TenderingProcess - ParticipationRequestReceptionPeriod - EndDate,ContractFolderStatus - TenderingProcess - ParticipationRequestReceptionPeriod - EndTime,ContractFolderStatus - TenderResult - AwardedTenderedProject - ProcurementProjectLotID
0,https://contrataciondelestado.es/sindicacion/P...,Id licitación: C. 2-2021; Órgano de Contrataci...,L'objecte del contracte és la renovació de tot...,2022-01-03T01:11:41.826+01:00,C. 2-2021,ADJ,https://contractaciopublica.gencat.cat/ecofin_...,Ajuntament de Sant Ramon,Entitats municipals de Catalunya,L'objecte del contracte és la renovació de tot...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 8128_3/2021; Órgano de Contrata...,Obras de restauración hidromorfológica del río...,2022-01-03T01:00:11.194+01:00,8128_3/2021,PUB,NaN,Pleno del Ayuntamiento,AYUNTAMIENTO DE MONREAL,Obras de restauración hidromorfológica del río...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 1000_0005-CP01-2021-000063; Órg...,Contrato del servicio de realización de labore...,2022-01-03T01:00:10.399+01:00,1000_0005-CP01-2021-000063,EV,NaN,El Director General de Comunicación y Relacion...,"Departamento de Presidencia, Igualdad, Función...",Contrato del servicio de realización de labore...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 1379/2020 4738; Órgano de Contr...,Obres de renovació de l'enllumenat públic a la...,2022-01-03T00:11:40.740+01:00,1379/2020 4738,EV,https://contractaciopublica.gencat.cat/ecofin_...,Ajuntament de Canet de Mar,Entitats municipals de Catalunya,Obres de renovació de l'enllumenat públic a la...,...,https://contractaciopublica.gencat.cat/ecofin_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 2021-44; Órgano de Contratación...,Subministre i la instal·lació fotovoltaica en ...,2022-01-03T00:11:40.696+01:00,2021-44,EV,https://contractaciopublica.gencat.cat/ecofin_...,Ajuntament de Valls,Entitats municipals de Catalunya,Subministre i la instal·lació fotovoltaica en ...,...,https://contractaciopublica.gencat.cat/ecofin_...,Enllac plec clausules tecniques.doc,https://contractaciopublica.gencat.cat/ecofin_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 1005_391-2021; Órgano de Contra...,Apoyo a la gestión del patrimonio filmográfico...,2021-12-31T01:00:14.946+01:00,1005_391-2021,PUB,NaN,Dirección General de Cultura-Institución Prínc...,"Departamento de Cultura, Deporte y Juventud",Apoyo a la gestión del patrimonio filmográfico...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 8165_3/2021; Órgano de Contrata...,Asistencia técnica para la prestación del serv...,2021-12-31T01:00:14.393+01:00,

A function to tidy up some things in the new `pd.DataFrame`

In [ ]:
# export
def post_process(input_df: pd.DataFrame) -> pd.DataFrame:
    
    res = df.copy()
    
    # ------------ ContractFolderStatus - TenderingProcess - TenderSubmissionDeadlinePeriod ------------
    
    new_column = f'ContractFolderStatus{nested_tags_separator}TenderingProcess{nested_tags_separator}TenderSubmissionDeadlinePeriod'
    
    # we don't want to inadvertently overwrite an existing column
    assert new_column not in res
    res[new_column] = pd.to_datetime(
        input_df[f'ContractFolderStatus{nested_tags_separator}TenderingProcess{nested_tags_separator}TenderSubmissionDeadlinePeriod{nested_tags_separator}EndDate']
        + 'T' +
        input_df[f'ContractFolderStatus{nested_tags_separator}TenderingProcess{nested_tags_separator}TenderSubmissionDeadlinePeriod{nested_tags_separator}EndTime'],
        format='%Y-%m-%dT%H:%M:%S'
    )
    
    # -------------------------------------------- updated ---------------------------------------------
    
    res['updated'] = pd.to_datetime(input_df['updated'], format='%Y-%m-%dT%H:%M:%S.%f%z')
    
    return res

In [ ]:
post_df = post_process(df)
post_df.head()

,id,summary,title,updated,ContractFolderStatus - ContractFolderID,ContractFolderStatus - ContractFolderStatusCode,ContractFolderStatus - LocatedContractingParty - BuyerProfileURIID,ContractFolderStatus - LocatedContractingParty - Party - PartyName - Name,ContractFolderStatus - LocatedContractingParty - ParentLocatedParty - PartyName - Name,ContractFolderStatus - ProcurementProject - Name,...,ContractFolderStatus - TechnicalDocumentReference - ID,ContractFolderStatus - TechnicalDocumentReference - Attachment - ExternalReference - URI,ContractFolderStatus - ProcurementProject - PlannedPeriod - StartDate,ContractFolderStatus - ProcurementProject - PlannedPeriod - EndDate,ContractFolderStatus - LocatedContractingParty - Party - PartyIdentification - ID,ContractFolderStatus - LocatedContractingParty - ParentLocatedParty - ParentLocatedParty - PartyName - Name,ContractFolderStatus - TenderingProcess - ParticipationRequestReceptionPeriod - EndDate,ContractFolderStatus - TenderingProcess - ParticipationRequestReceptionPeriod - EndTime,ContractFolderStatus - TenderResult - AwardedTenderedProject - ProcurementProjectLotID,ContractFolderStatus - TenderingProcess - TenderSubmissionDeadlinePeriod
0,https://contrataciondelestado.es/sindicacion/P...,Id licitación: C. 2-2021; Órgano de Contrataci...,L'objecte del contracte és la renovació de tot...,2022-01-03 01:11:41.826000+01:00,C. 2-2021,ADJ,https://contractaciopublica.gencat.cat/ecofin_...,Ajuntament de Sant Ramon,Entitats municipals de Catalunya,L'objecte del contracte és la renovació de tot...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-17 14:00:00
1,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 8128_3/2021; Órgano de Contrata...,Obras de restauración hidromorfológica del río...,2022-01-03 01:00:11.194000+01:00,8128_3/2021,PUB,NaN,Pleno del Ayuntamiento,AYUNTAMIENTO DE MONREAL,Obras de restauración hidromorfológica del río...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-01-22 23:30:00
2,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 1000_0005-CP01-2021-000063; Órg...,Contrato del servicio de realización de labore...,2022-01-03 01:00:10.399000+01:00,1000_0005-CP01-2021-000063,EV,NaN,El Director General de Comunicación y Relacion...,"Departamento de Presidencia, Igualdad, Función...",Contrato del servicio de realización de labore...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 1379/2020 4738; Órgano de Contr...,Obres de renovació de l'enllumenat públic a la...,2022-01-03 00:11:40.740000+01:00,1379/2020 4738,EV,https://contractaciopublica.gencat.cat/ecofin_...,Ajuntament de Canet de Mar,Entitats municipals de Catalunya,Obres de renovació de l'enllumenat públic a la...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-01-02 23:59:00
4,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 2021-44; Órgano de Contratación...,Subministre i la instal·lació fotovoltaica en ...,2022-01-03 00:11:40.696000+01:00,2021-44,EV,https://contractaciopublica.gencat.cat/ecofin_...,Ajuntament de Valls,Entitats municipals de Catalunya,Subministre i la instal·lació fotovoltaica en ...,...,Enllac plec clausules tecniques.doc,https://contractaciopublica.gencat.cat/ecofin_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-01-02 23:59:00


In [ ]:
post_df.dtypes

id                                                                                                                                                         object
summary                                                                                                                                                    object
title                                                                                                                                                      object
updated                                                                                                                      datetime64[ns, pytz.FixedOffset(60)]
ContractFolderStatus - ContractFolderID                                                                                                                    object
ContractFolderStatus - ContractFolderStatusCode                                                                                                            object
ContractFolderStatus - Locat

In [ ]:
# hide
import nbdev.export
nbdev.export.notebook2script('10_xml.ipynb')

Converted 10_xml.ipynb.
